In [ ]:
# Часть I - генерация изначального хронометража

import gpxpy # Анализ gpx файлов
import gpxpy.gpx 
import os
import datetime
from datetime import timedelta
import speech_recognition as sr # Распознование голоса
from docx import Document # Docx документы
from docx.shared import Pt
from docx.shared import RGBColor
from pydub import AudioSegment #
from exif import Image # Парсер Расширенного формат файлов
import pytz # Часовые пояса
import functools
import math

#**********#
#**Config**#
#**********#

tz = pytz.timezone("Europe/Moscow") # Часовой пояс
project = "." # Относительный адрес папки с походным контентом
gpx_dir = project + "/tracks" # Относительный адресс папки с треком
points_dir = project + "/points" # Относительный адресс папки с точками
path_dict = project + "/records" # Относительный адресс непустой папки с диктофонными записями
path_photo = project + "/photos" # Относительный адресс непустой папки с фото
result = project + "/report.docx" # Файл для сохранения хронометража

delay = 300 # Минимальное время остановки в секундах. 
            # Если время между двумя точками превышает этот интервал - то это отмечается в хронометраже
nd_hour = 3 # В три часа ночи начинается трек нового дня
show_seconds = True # Отображать секунды в хронометраже?
use_audio_parser = True # Использовать распознование голоса?
clean = False # Чистить трек?

#**********#
#***Code***#
#**********#

r = sr.Recognizer() # Экземпляр распознователя голоса
d = Document() # Word-документ с хронометражом 
files = os.listdir(path_dict)
files = [os.path.join(path_dict, file) for file in files]
files = [file for file in files if os.path.isfile(file)]
files.sort(key=os.path.getmtime) # Диктофонные записи

photo = os.listdir(path_photo)
photo = [os.path.join(path_photo, file) for file in photo]
photo = [file for file in photo if os.path.isfile(file)]

def strip_tmpl(dt):
    try:
        return datetime.datetime.strptime(dt, '%d:%m:%Y %H:%M:%S')
    except:
        try:
            return datetime.datetime.strptime(dt, '%Y:%m:%d %H:%M:%S')
        except: 
            return datetime.datetime.fromtimestamp(0)
    
photo = list(map(lambda f: (f, strip_tmpl(Image(open(f, 'rb')).datetime_original)), photo))
photo.sort(key=(lambda t: t[1])) #Фотографии

skip = [] # Днёвки в формате номеров дней - из километража исключаются

parts = os.listdir(gpx_dir)
parts = [(os.listdir(os.path.join(gpx_dir, part)), part) for part in parts] if "part1" in parts else [(parts, "")]
parts = [[gpxpy.parse(open(os.path.join(gpx_dir, part_name, gpx_file),'r')) for gpx_file in part] for part, part_name in parts]
def join(a, v):
    for track in v.tracks:
        a.tracks.append(track)
    for waypoint in v.waypoints:
        if not waypoint.time is None:
            a.waypoints.append(waypoint)
    return a
parts = [functools.reduce(join, part, gpxpy.gpx.GPX()) for part in parts]
    # Каждая часть это gpx из нескольких треков. Окончание одного трека - это начало другого, 
    # однако части между собой так не соединяются

points = os.listdir(points_dir)
points = [gpxpy.parse(open(os.path.join(points_dir, gpx_file),'r')) for gpx_file in points]
points = functools.reduce(join, points, gpxpy.gpx.GPX())
points = points.waypoints
points.sort(key=(lambda t: t.time))

    
prpoint = None # Предыдущая точка
dbpoint = None # Точка начала дня

dayn = 0 # Счётчик дней

dist = 0 # Глобальная дистанция
ddist = 0 # Дневная дистанция
high = 0 # Глобальный набор высоты 
dhigh = 0 # Дневной набор высоты
low = 0 # Глобальный сброс высоты
dlow = 0 # Дневной сброс высоты

audio_formats = [(".mp3", "mp3"), (".m4a", "aac"), (".wma", "asf"), (".wav", "wav")] # Поддерживаемые форматы аудио

# Широту/Долготу в строку
def get_cs(coord):
    return "{0:}° {1}' {2:.2f}''".format(math.trunc(coord),math.trunc(coord*60)%60,round(coord*3600,2)%60)

# время следущей диктофонной
def dicttime():
    return datetime.datetime.fromtimestamp(os.path.getmtime(files[0])) #+ timedelta(seconds=0, hours=2)

# Точку в строку
def get_csp(point):
    return "{0} {1}, {2} {3}".format('N' if point.latitude>=0 else 'S',get_cs(point.latitude),
                                     'E' if point.longitude>=0 else 'W',get_cs(point.longitude))
def zapis(n):
    if 10 < n%100 < 20 or 4 < n%10 < 9 or n%10 == 0:
        return "записей"
    if 1 < n%10 < 5:
        return "записи"
    return "запись"

# секунды
def second(p):
    return p.time.astimezone(tz).second

# минуты
def minute(p):
    return p.time.astimezone(tz).minute

# часы
def hour(p):
    return p.time.astimezone(tz).hour
# время простановки точки
def point_time(p):
    return "{0}:{1:02d}".format(hour(p),minute(p)) if not show_seconds else "{0}:{1:02d}:{2:02d}".format(hour(p),minute(p),second(p))

# время    
def time(tm):
    return "{0}:{1:02d}".format(tm.hour,tm.minute) if not show_seconds else "{0}:{1:02d}:{2:02d}".format(tm.hour,tm.minute,tm.second)

# Распознать аудиозапись
def getdictstr_rec(audio):
    try:
        return r.recognize_google(audio, language="Ru-ru")#recognize_wit(audio, key=wit_key)
    except sr.UnknownValueError:
        return "Запись"
    except sr.RequestError as e:
        #print(e)
        return str(e)
        #return getdictstr_rec(audio)

# Диктофонную запись в строку
def getdictstr(file):
    if use_audio_parser:
        aformat = None
        fformat = os.path.splitext(file)[1].lower()
        for e_f in audio_formats:
            if e_f[0] == fformat:
                aformat = e_f[1]
                break
        if aformat == None:
            raise Exception("Unsupported audio format: " + fformat)
        segment = AudioSegment.from_file(os.path.abspath(file), aformat)
        segment.export("temp.wav", format="wav")
        with sr.AudioFile("temp.wav") as source:
            audio = r.record(source)
        return getdictstr_rec(audio)
    else:
        return os.path.basename(file)
        
# Печать в word
def wordprint(s, b = False, c = False, color = None):
    print(s)
    p = d.add_paragraph()
    r = p.add_run(s)
    r.bold = b
    r.italic = c
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    font.color.rgb = color
    return p

# Альтернативная печать в word
def wordprint2(s1, s2, color = None):
    p = wordprint(s1, b = True, color = color)
    r = p.add_run(s2)
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    font.color.rgb = color
    print(s2)
    return p

# Добавить в хронометраж диктофонные записи к данной точке
def printdict(p, f_all=True):
    if len(files) > 0:
        tm = dicttime()
        while (len(files) > 0 
               and tz.localize(tm) < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour 
                    and tm.day == p.time.astimezone(tz).day)):
            pstr = getdictstr(files[0])
            wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)"
                       .format(time(tm),ddist/1000.0,dist/1000.0), '\u00A0'+pstr)
            #print(files[0])
            files.remove(files[0])
            if len(files) > 0:
                tm = dicttime()
    return

# Добавить в хронометраж имена фото-файлов к данной точке
def printphoto(p, f_all=True):
    if len(photo) > 0:
        tm = photo[0][1]
        while (len(photo) > 0 
               and tz.localize(tm) < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour
                    and tm.day == p.time.astimezone(tz).day)):
            pstr = os.path.basename(photo[0][0])
            wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)"
                       .format(time(tm),ddist/1000.0,dist/1000.0), '\u00A0'+pstr)
            d.add_picture(photo[0][0])
            photo.remove(photo[0])
            if len(photo) > 0:
                tm = photo[0][1]
    return

# Добавить в хронометраж имена путевых точек к данной точке
def printpoints(p, f_all=True):
    if len(points) > 0:
        tm = points[0].time
        while (len(points) > 0 
               and tm < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour
                    and tm.day == p.time.astimezone(tz).day)):
            pstr = 'Точка {0} -> {1}.'.format(points[0].name, get_csp(points[0]))
            wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)"
                       .format(point_time(points[0]),ddist/1000.0,dist/1000.0), '\u00A0'+pstr)
            points.remove(points[0])
            if len(points) > 0:
                tm = points[0].time
    return

# Вернуть диктофонные записи и их число в данной точке
def dictn(p):
    count = 0
    pstr = ""
    if len(files) > 0:
        tm = dicttime()
        while len(files) > 0 and tz.localize(tm) < p.time.astimezone(tz):
            pstr += getdictstr(files[0]) + ". "
            files.remove(files[0])
            count+=1
            if len(files) > 0:
                tm = dicttime()
    return (pstr, count) #if use_audio_parser else (str(count) + " " + zapis(count) + ".", count)

# Вернуть список файлов с фото и их число в данной точке
def photon(p):
    count = 0
    pl = []
    if len(photo) > 0:
        tm = photo[0][1]
        while len(photo) > 0 and tz.localize(tm) < p.time.astimezone(tz):
            pl.append(photo[0][0])
            count+=1
            photo.remove(photo[0])
            if len(photo) > 0:
                tm = photo[0][1]
    return (pl, count)

# Вернуть имена путевых точек и их число в данной точке
def pointn(p):
    count = 0
    pstr = ""
    if len(points) > 0:
        tm = points[0].time
        while len(points) > 0 and tm < p.time.astimezone(tz):
            pstr += 'Точка {0} -> {1}.'.format(points[0].name, get_csp(points[0]))
            count += 1
            points.remove(points[0])
            if len(points) > 0:
                tm = points[0].time
    return (pstr + ".", count)

# Печать путевых точек, фото и диктофона к данной точке
def printdnf(p, f_all=True):
    printpoints(p, f_all = f_all)
    printdict(p, f_all = f_all)
    printphoto(p, f_all = f_all)
    return

# dictn + photon + pointn
def dnfnpn(p):
    (t0, t1) = dictn(p)
    (t2, t3) = photon(p)
    (t4, t5) = pointn(p)
    return (t0, t1, t2, t3, t4, t5)

for gpx in parts:
    prpoint = None
    for track in gpx.tracks:
        for segment in track.segments:
            # Чистим трек от точек проставленных на скорости <= 2 км/ч
            if clean:
                pns = list()
                pn = 0
                for p1, p2 in zip(segment.points[:-1], segment.points[1:]):
                    if (p1.distance_3d(p2)/p1.time_difference(p2)*3.6) <= 2.0:
                        pns.append(pn)
                    pn+=1
                for p in pns[::-1]:
                    segment.remove_point(p)
            
            # Обработка дней похода
            for point in segment.points:
                if prpoint == None:
                    prpoint = point
                if ((dbpoint == None) 
                    or (point.time.astimezone(tz).day != dbpoint.time.astimezone(tz).day 
                        and hour(point) >= nd_hour)):
                    delta = (point.time.date() - prpoint.time.date()).days
                    dayn += 1 if delta <= 1 else delta
                    if dbpoint != None:
                        wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
                                   .format(point_time(prpoint),ddist/1000.0,dist/1000.0), "Окончание ходового дня.", color = RGBColor(0, 176, 80))
                        printdnf(point, f_all = False)
                        wordprint("Пройденное расстояние: {0:.2f} км".format(ddist/1000.0), c=True)
                        wordprint("Набор высоты: {0:.1f} м".format(dhigh), c=True)
                        wordprint("Сброс высоты: {0:.1f} м".format(dlow), c=True)
                        wordprint("Высота ночёвки: {0:.1f} м".format(point.elevation), c=True)
                        wordprint("Координаты точки ночёвки: {0}".format(get_csp(point)), c=True)
                        wordprint("")
                    else:
                        printdnf(point)
                        wordprint("")
                    wordprint("День {0}. {1}".format(dayn,point.time.astimezone(tz).date()),b=True,c=True)
                    wordprint("МС{0} \u2014 МС{1}".format(dayn-1,dayn),c=True)
                    if dayn in skip:
                        wordprint("Днёвка.")
                    dbpoint = point
                    ddist = 0
                    dhigh = 0
                    dlow = 0
                    printdnf(point)
                    wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0".format
                        (point_time(point),ddist/1000.0,dist/1000.0), "Начало ходового дня.", color = RGBColor(0, 176, 80))
                else:
                    if not dayn in skip:
                        dist += prpoint.distance_3d(point)
                        if point.elevation-prpoint.elevation > 0:
                            high += point.elevation-prpoint.elevation
                        else:
                            low += prpoint.elevation-point.elevation
                    ddist += point.distance_3d(prpoint)
                    if point.elevation-prpoint.elevation > 0:
                            dhigh += point.elevation-prpoint.elevation
                    else:
                            dlow += prpoint.elevation-point.elevation
                    if point.time_difference(prpoint) > delay:
                        (dicts, x, photos, _y, waypoints, z) = dnfnpn(point)
                        wordprint2("{0}-{1}\u00A0({2:.2f}\u00A0км/{3:.2f}\u00A0км)\u00A0"
                                   .format(point_time(prpoint),point_time(point),ddist/1000.0,dist/1000.0),
                                           ("" if z == 0 else waypoints)+("" if x == 0 else dicts))
                        for p in photos:
                            wordprint(os.path.basename(p))
                            d.add_picture(p)
                            
                    else:
                        printdnf(point)
                prpoint = point
  
 # Последний день с итогами
if dbpoint != None:
    wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
               .format(point_time(point),ddist/1000.0,dist/1000.0), "Окончание ходового дня.", color = RGBColor(0, 176, 80))
    wordprint("Пройденное расстояние: {0:.2f} км".format(ddist/1000.0), c=True)
    wordprint("Набор высоты: {0:.1f} м".format(dhigh), c=True)
    wordprint("Сброс высоты: {0:.1f} м".format(dlow), c=True)
    wordprint("Высота ночёвки: {0:.1f} м".format(point.elevation), c=True)
    wordprint("Координаты точки ночёвки: {0}".format(get_csp(point)), c=True)
    wordprint("")
    wordprint("\nИтого:", c=True)
    wordprint("    Дней: {0}".format(dayn), c=True)
    wordprint("    Пройденное расстояние: {0:.2f} км".format(dist/1000.0), c=True)
    wordprint("    Набор высоты: {0:.1f} м".format(high), c=True)
    wordprint("    Сброс высоты: {0:.1f} м".format(low), c=True)

# wordprint("")
# for waypoint in points.waypoints:
#    wordprint('{0}.{1} ({2}) Точка {2} -> ({3},{4})'.format(waypoint.time.day, waypoint.time.month, point_time(waypoint), waypoint.name, waypoint.latitude, waypoint.longitude))

# Печатаем результат в файл
d.save(result)

In [ ]:
# Часть II - подсчёт статистики.
# На вход подаётся раскрашенный документ.
# Зелёным цветом выделяются сообщения о начале и окончании ходового дня.
# Эти сообщения вставляются автоматически на первом этапе.
# Синим - сообщения о смене покрытия, наличие ЛП, а также начале/окончании радиальных участков и ПП.
# Важно выделять сообщения вместе с меткой времени/километража, а также заканчивать каждое сообщение точкой.
# Необходимо также указать самое первое покрытие сразу после начала первого ходового дня.
# Цвета в формате RGB вычисляются по формуле B-G-R>=50 (для синего цвета).
# Указание времени поддерживается как с точностью до минут, так и с точностью до секунд.
# Всё что написано курсивом из хронометража удаляется! 
# Не используйте курсив для оформления покрытий, при трансляции в формат велотрекс покрытия автоматически выделяются жирным курсивом 


# Форматы сообщений: 
# 7:55 (0.00 км/303.80 км) Начало ходового дня. 
#   - изменения не допускаются.
# 17:28 (88.48 км/392.28 км) Окончание ходового дня. 
#   - изменения не допускаются.
# 8:36 (0.62 км/304.42 км) Начало радиального участка. 
#   - изменения не допускаются.
# 8:44 (2.93 км/306.73 км) Окончание радиального участка. 
#   - изменения не допускаются.
# 8:36 (0.62 км/304.42 км) Начало ПП1. 
#   - Указывается номер ПП.
# 8:36 (0.62 км/304.42 км) Окончание ПП1. 
#   - Указывается номер ПП.
# 8:36 (0.62 км/304.42 км) Покрытие – асфальт хорошего качества. 
#   - вместо фразы асфальт хорошего качества необходимо вставить нужное покрытие.
#   - виды покрытий, в соответствии с МКТВМ, описаны в файле surfs.json, при необходимости можно добавить свои
#   - внимание! есть поддержка длинного тире из ворда.
# 8:11-8:17 (0.79 км/449.69 км) Покрытие – грунт разбитый/грунт с ТП (30/70).
#   - формат смешанных покрытий указывается через "/", в скобках указывается соотношение
# 15:04:48 (32.33 км/71.83 км) ЛП - брод (н/к).
#   - вместо фразы брод (н/к) необходимо вставить нужное ЛП.
#   - виды ЛП, в соответствии с МКТВМ, описаны в файле lp.json, при необходимости можно добавить свои
# 15:04:48 (32.33 км/71.83 км) ЛП - подъём (н/к), протяжённость 550 м.
#   - Для протяжённых ЛП необходимо указать протяжённость в метрах, строго как на шаблоне выше.

# Для интеграции с velotrex.ru:
#   Необходимо записать дату начала похода в переменную start_date
#   Необходимо записать ссылку на публичную папку с фото в облаке в переменную link
#   Имена фото должны быть без пробелов и могут содержать буквы, цифры, "_", "-".
#   Важно, при трансляции в формат velotrex время начала и окончания ходового дня совпадает с защищёнными событиями,
#     а соответственно записи с таким временем не загружаются на сайт. Для решения этой проблемы следует изменить время 
#     таких записей, например добавив +1 сек. Также на велотрекс не стоит загружать записи датой/временем до первой и после
#     последней точки всего трека.

import os
import datetime
from docx import Document # Docx документы
from docx.shared import Pt
from exif import Image # Парсер Расширенного формат файлов
import pytz # Часовые пояса
import functools
import re
from enum import Enum
import json
import pyexcel
import datetime
from datetime import timedelta
import time

#**********#
#**Config**#
#**********#

project = "." # Относительный адрес папки с походным контентом
result_p = project + "/report.docx" # Файл хронометража
result_t = project + "/report_tabs.docx" # Результат с таблицами

start_date = datetime.date(1111, 11, 11) # start date: yyyy, mm, dd
link = "" # Ссылка на публичную папку с фото в облаке без завершающего '/'
photo_mask = r'(?:[-_\d\w]+)\.[Jj][Pp][Gg]' # Маска для имён фото-файлов
dayn = 1 # номер стартового дня

#**********#
#***Code***#
#**********#

color_threshold = 50
d_source = Document(result_p)
d = Document()
days = []
daylen = ""
surfs = ""
used_photos = ["mkdir used_photos\n"]

tmessage = r'(\d{1,2}:\d{2}(?::\d{2})?)(?:-\d{1,2}:\d{2}(?::\d{1,2})?)?[\xA0 ]\((\d+\.\d{1,2})[\xA0 ]км/\d+\.\d{1,2}[\xA0 ]км\)[\xA0 ]?'

message = r'\d{1,2}:\d{2}(?::\d{2})?(?:-\d{1,2}:\d{2}(?::\d{1,2})?)?[\xA0 ]\((\d+\.\d{1,2})[\xA0 ]км/\d+\.\d{1,2}[\xA0 ]км\)[\xA0 ]?((?:\w|-|–|,|\s|\d|\(|\)|/)+)\.'
surf = r'Покрытие [-–] ((?:\w|,|\s)+)(?:/((?:\w|,|\s)+) \((\d{1,2})/(\d{1,2})\))?'
pp_start = r'Начало ПП(\d+)'
pp_end = r'Окончание ПП(\d+)'
lp = r'ЛП [-–] ((?:\w|-|–|\s|\(|\)|/)+)(?:, протяжённость (\d+) м)?'
surfs_base = json.load(open("surfs.json"))
lp_base = json.load(open("lp.json"))
day_start = "Начало ходового дня"
day_end = "Окончание ходового дня"

def addSecs(tm, secs):
    fulldate = datetime.datetime(100, 1, 1, tm.hour, tm.minute, tm.second)
    fulldate = fulldate + datetime.timedelta(seconds=secs)
    return fulldate.time()

def subSecs(tm, secs):
    fulldate = datetime.datetime(100, 1, 1, tm.hour, tm.minute, tm.second)
    fulldate = fulldate - datetime.timedelta(seconds=secs)
    return fulldate.time()

dayn2 = 1
phn = 1
def sub_link(m):
    global phn
    tmpl = "<a target=\'_blank\' href=\'"+link+"/"+m.group(0)+"\'>Фото "+str(dayn2)+"-"+str(phn)+"</a>"
    used_photos.append("cp \"" + m.group(0) + "\" \"used_photos/" + m.group(0) + "\"\n")
    phn+=1
    return tmpl

def sub_b_i(m):
    tmpl = "<b><i>"+m.group(0)+"</i></b>"
    return tmpl

vt_surfs = []
vt_lp = []

current_surf = ([], [], False, None)
pps = dict()
ndhour = datetime.time(3, 0, 0)
date = start_date
sf_date = start_date

class SQ(Enum):
    RAD = -1.0
    UNKNOWN = 0.0
    HIGH = 0.8
    GOOD = 1.11
    MEDIUM = 1.51
    LOW = 1.91
    ULTRA_LOW = 2.41
    LP = 4.0

def find_surf_k_simple(s):
    if s[1]:
        return SQ.RAD.value
    for sf in surfs_base:
        if s[0] in sf["names"]:
            return sf["k"]
    return 0
    
def find_surf_k(s):
    if len(s[0]) == 1:
        return find_surf_k_simple((s[0][0], s[2]))
    else:
        k1 = find_surf_k_simple((s[0][0], s[2]))
        k2 = find_surf_k_simple((s[0][1], s[2]))
        if k1 == 0 or k2 == 0:
            return 0
        return (k1 * s[1][0] + k2 * s[1][1])/100.0

def find_lp(name):
    for lp in lp_base:
        if name in lp["names"]:
            return lp
    print("!!!ЛП: " + name + "not found")
    return 0

def quality_to_string(q):
    if q == SQ.RAD:
        return "радиального/неактивного"
    if q == SQ.LP:
        return "ЛП"
    if q == SQ.ULTRA_LOW:
        return "сверхнизкого"
    if q == SQ.LOW:
        return "низкого"
    if q == SQ.MEDIUM:
        return "среднего"
    if q == SQ.GOOD:
        return "хорошего"
    if q == SQ.HIGH:
        return "высокого"
    return "неизвестного"

def quality_to_code(q):
    if q == SQ.RAD:
        return 6
    if q == SQ.LP:
        return 0
    if q == SQ.ULTRA_LOW:
        return 5
    if q == SQ.LOW:
        return 4
    if q == SQ.MEDIUM:
        return 3
    if q == SQ.GOOD:
        return 2
    if q == SQ.HIGH:
        return 1
    return 0
    
def k_to_quality(k):
    if k == SQ.RAD.value:
        return SQ.RAD
    if k >= SQ.LP.value:
        return SQ.LP
    if k >= SQ.ULTRA_LOW.value:
        return SQ.ULTRA_LOW
    if k >= SQ.LOW.value:
        return SQ.LOW
    if k >= SQ.MEDIUM.value:
        return SQ.MEDIUM
    if k >= SQ.GOOD.value:
        return SQ.GOOD
    if k >= SQ.HIGH.value:
        return SQ.HIGH
    return SQ.UNKNOWN

def time_format(t):
    if len(t) == 4 or len(t) == 7:
        t = "0" + t
    if len(t) == 5:
        t += ":30"
    return t
        
def date_format(d):
    return str(d.day).zfill(2) + "." + str(d.month).zfill(2) + "." + str(d.year)

# разделение покрытий с использованием точек в качестве разделителей.
def split_surfs(surfs):
    res = []
    p0 = 0
    p1 = surfs.find('.', p0) + 1
    while p1 != 0:
        p2 = surfs.find('.', p1) + 1
        p3 = surfs.find('.', p2) + 1
        res.append(surfs[p0:p3])
        print(surfs[p0:p3])
        time.sleep(0.001)
        p0 = p3
        p1 = surfs.find('.', p0) + 1
    return res

# Выделение типа и точки начала покрытия, радиальные участки также считаются как смена покрытия
def parse_surf(s):
    global current_surf
    global pps
    global sf_date

    try:
        ps = re.search(message, s.strip())
        ctm = re.search(tmessage, s.strip())
        t = time_format(ctm[1])
        
        if ps[2] == "Начало радиального участка":
            current_surf = (current_surf[0], current_surf[1], True, current_surf[3])
            q = quality_to_code(k_to_quality(find_surf_k(current_surf)))
            vt_surfs.append((date_format(sf_date), t, q))
        elif ps[2] == "Окончание радиального участка":
            current_surf = (current_surf[0], current_surf[1], False, current_surf[3])
            q = quality_to_code(k_to_quality(find_surf_k(current_surf)))
            vt_surfs.append((date_format(sf_date), t, q))
        else:
            m = re.search(pp_start, ps[2].strip())
            if not (m is None):
                pps[m[1]] = dict()
                current_surf = (current_surf[0], current_surf[1], current_surf[2], m[1])
            else:
                m = re.search(pp_end, ps[2].strip())
                if not (m is None):
                    current_surf = (current_surf[0], current_surf[1], current_surf[2], None)
                else:
                    m = re.search(lp, ps[2].strip())
                    if not m is None:
                        print("ЛП: " + m[1])
                        vt_lp.append((date_format(sf_date), t, find_lp(m[1])["code"], None if m[2] is None else int(m[2])))
                    else:
                        m = re.search(surf, ps[2].strip())
                        if m[2] is None:
                            current_surf = ([m[1]], [100.0], current_surf[2], current_surf[3])
                            q = quality_to_code(k_to_quality(find_surf_k(current_surf)))
                            vt_surfs.append((date_format(sf_date), t, q))
                        else:
                            current_surf = ([m[1], m[2]], list(map(float, [m[3], m[4]])), current_surf[2], current_surf[3])
                            q = quality_to_code(k_to_quality(find_surf_k(current_surf)))
                            vt_surfs.append((date_format(sf_date), t, q))
        res = (float(ps[1]), current_surf)
    except Exception as err:
        print("!!!" + s)
        raise err
    return res

# Разделение дня на учачстки по покрытиям
def split_day(day):
    #print(day)
    global sf_date
    surfs = split_surfs(day[1])
    #print(surfs)
    surfs = list(map(parse_surf, surfs))
    sf_date += timedelta(days = 1)
    #print()
    return (day[0], surfs)

def z_or_val(d, k):
    return 0.0 if d.get(k) is None else d[k]

# Объединение однотипных покрытий за день
def stack_day(day):
    d = dict()
    global current_surf
    global pps
    current_dist = 0
    for surf in day[1]:
        if current_surf != ([], [], False, None):
            for (sf, sz) in zip(current_surf[0], current_surf[1]):
                dist_d = z_or_val(d, (sf, current_surf[2]))
                s_len = (surf[0] - current_dist)/100.0*sz
                d[(sf, current_surf[2])] = dist_d + s_len
                if not (current_surf[3] is None):
                    dist_pp = z_or_val(pps[current_surf[3]], (sf, current_surf[2]))
                    pps[current_surf[3]][(sf, current_surf[2])] = dist_pp + s_len
                    
        else:
            print("WARNING: missed starting surf!")
        current_dist, current_surf = surf
    for (sf, sz) in zip(current_surf[0], current_surf[1]):
        dist_d = z_or_val(d, (sf, current_surf[2]))
        s_len = (day[0] - current_dist)/100.0*sz
        d[(sf, current_surf[2])] = dist_d + s_len
        if not (current_surf[3] is None):
            dist_pp = z_or_val(pps[current_surf[3]], (sf, current_surf[2]))
            pps[current_surf[3]][(sf, current_surf[2])] = dist_pp + s_len
    d = dict(sorted(d.items(), key=lambda x: x[0]))
    return (day[0], d)

def stack_quality(surfs):
    q_surfs = dict()
    for surf, dist in surfs.items():
        k = find_surf_k_simple(surf)
        q = k_to_quality(k)
        q_surfs[q] = z_or_val(q_surfs, q) + dist
    return q_surfs

# Печать в ворд
def wordprint(s, b = False, c = False, color = None):
    print(s)
    p = d.add_paragraph()
    r = p.add_run(s)
    r.bold = b
    r.italic = c
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    font.color.rgb = color
    return p

def print_stats(d):
    surfs = []
    rad = 0.0
    q_surfs = stack_quality(d)
    for surf, dist in d.items():
        if dist != 0.0:
            surfs.append("{0}{1} – {2:.2f} км".format(surf[0], " (радиально)" if surf[1] else "", dist))
        if surf[1]:
            rad += dist
    
    wordprint("из них" + ("" if rad == 0.0 else " радиально {0:.2f} км".format(rad)), c=True)
    for q, dist in q_surfs.items():
        if dist != 0.0:
            if q == SQ.RAD:
                pass
            elif q == SQ.LP:
                wordprint("по ЛП – {0:.2f} км".format(dist), c=True)
            else:
                wordprint("по дорогам {0} качества – {1:.2f} км".format(quality_to_string(q), dist), c=True)

    wordprint("", c=True)
    for surf in surfs:
        wordprint(surf, c=True)
    wordprint("", c=True)
    

# Печать статистики за день в ворд
def print_day(d, total = False):
    if total:
        wordprint("Итого".format(dayn), b=True)
    else:
        wordprint("День {0}".format(dayn), b=True)
    wordprint("Пройденное расстояние: {0:.2f} км".format(d[0]), c=True)
    print_stats(d[1])
    
    if total:
        for num, pp in pps.items():
            wordprint("ПП{0}".format(num), b=True)
            pp_len = functools.reduce(lambda acc, p: p[1]+acc, pp.items(), 0)
            wordprint("Протяжённость: {0:.2f} км".format(pp_len), c=True)
            print_stats(pp)
    return

def add_day(d1, d2):
    d = d1[1]
    for surf, dist in d2[1].items():
        if d.get(surf) is None:
            d[surf] = dist
        else:
            d[surf] += dist
    d = dict(sorted(d.items(), key=lambda x: x[0]))
    return (d1[0] + d2[0], d)

acc = ""

for p in d_source.paragraphs:
    acc+="\n"
    for run in p.runs:
        acc+= "" if run.italic else run.text
        #print(p.style.font.color.rgb)
        (r, g, b) = (0, 0, 0)
        if not run.font.color.rgb is None:
            (r, g, b) = run.font.color.rgb
        elif not run.style.font.color.rgb is None:
            (r, g, b) = run.style.font.color.rgb
        elif not p.style.font.color.rgb is None:
            (r, g, b) = p.style.font.color.rgb
        if b == r == g == 0 or b - r - g >= color_threshold:
            if daylen != "":
                try:
                    dl = re.search(message, daylen.strip())
                    if dl[2] == day_start:
                        surfs = ""
                    elif dl[2] == day_end:
                        days.append((float(dl[1]), surfs))
                except Exception as err:
                    print("!!" + daylen)
                    print(err)
                daylen = ""
            if b - r - g >= color_threshold:
                surfs += run.text
        if g - b - r >= color_threshold:
            daylen += run.text

acc = list(filter(lambda s: s != '', acc.split('\n')))
def join(a, v):
    m = re.search(tmessage, v.strip())
    global date
    global dayn2
    global phn
    if m is None:
        a[0][2] += "<br>" + re.sub(photo_mask, sub_link, v.strip())
        return a
    else:
        t = time_format(m[1]) 
        tm = datetime.time.fromisoformat(t)
        if (tm < datetime.time.fromisoformat(a[0][1]) and m[2] == "0.00"):
            date += timedelta(days = 1)
            dayn2 += 1
            phn = 1
        v = re.sub(photo_mask, sub_link, v.strip())
        v = re.sub(surf, sub_b_i, v)
        v = re.sub(lp, sub_b_i, v)
        d = date_format(date)
        if d == a[0][0] and t == a[0][1]:
            a[0][2] += "<br>" + v[len(m[0]):]
            return a
        return [[d, t, v[len(m[0]):], None, None, None, None]] + a

acc = functools.reduce(join, acc, [[date_format(date), "03:00:00", "", None, None, None, None]])
acc = acc[::-1]

days = list(map(split_day, days))
currnet_surf = ([], [], False,None)
days = list(map(stack_day, days))
total = (0.0, dict())
for day in days:
    print_day(day)
    total = add_day(total, day)
    dayn += 1
print_day(total, total=True)
print(vt_lp)
print(vt_surfs)


def add_sf_lp(e):
    sf = next((x for x in vt_surfs if x[0] == e[0] and x[1] == e[1]), None)
    lp = next((x for x in vt_lp if x[0] == e[0] and x[1] == e[1]), None)
    if not (sf is None):
        vt_surfs.remove(sf)
        e[3] = sf[2]
    if not (lp is None):
        vt_lp.remove(lp)
        e[5] = lp[2]
        e[6] = lp[3]
    #print(e)
    return e
        
acc = list(map(add_sf_lp, acc))
acc = acc[1:]
#print(acc)
head = [["№ п/п", "Дата", "Время", "Описание события", "покрытие_код", "Метеоданные", "ЛП_код", "ЛП_длинна"]]

for i in range(0, len(acc)):
    acc[i] = [i+1] + acc[i]
    acc[i][1] = datetime.datetime.strptime(acc[i][1], '%d.%m.%Y').date()

records_num = 80

for i in range(0, int(len(acc)/records_num) + 1):
    data = head + acc[i*records_num:(i+1)*records_num]
    name = project + "/velotrex"+str(i)+".xls"
    data = pyexcel.Sheet(data)
    data.name_columns_by_row(0)
    data.column.format("Дата", date)
    data.save_as(name)
    
ph = open(project + "/used_photos.sh", "w")
ph.writelines(used_photos)
ph.close()
ph = open(project + "/used_photos.ps1", "w")
ph.writelines(used_photos)
ph.close()
d.save(result_t)